In [ ]:
# Import

import pandas as pd
import tweepy

In [ ]:
# Keys

ACCESS_TOKEN = "1048198054044557313-7Gg2KsT5BdXMms2YYUSMBDJcvqpz50"
ACCESS_TOKEN_SECRET = "00LmyoLAHAcPpHNNaqNWinUj3LtRMeOXyQ9tlsyxWwDBf"
CONSUMER_KEY = "SH0QQe5G2qVyUfWiUQ0ziNE0l"
CONSUMER_SECRET = "TIVBEbOiagfoabCVoOg3wEed5QoefrwZUl57tRIIC5bAraWz5x"

# BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAP1PjAEAAAAAVNecFA9PZ8bDLjTNhcZJvOTWs2A%3DnLM6ySxCM3RzcH35ZMiFAZYaQiLXfZda9uy4trNIYeXR8ML67Q"

In [ ]:
# Auth

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET) 
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET) 
api = tweepy.API(auth,wait_on_rate_limit=True)

# client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

In [ ]:
# Data

city_data = pd.read_csv('~/Documents/Practice/NEW_ORLEANS.csv', header=None, dtype=str)
Columns = ["Tweet", "Tweet-id", "Time(Month-Year)"]
city_data.columns = Columns

In [ ]:
# Prepare mapping for tweet-id with it's MonthYear

tweets_dict = {}
for index, row in city_data.iterrows():
    if index == 0:
        continue
    tweets_dict[row['Tweet-id']] = row['Time(Month-Year)']

In [ ]:
# Store Tweet-ids

tweet_ids = list(tweets_dict.keys())

In [ ]:
# users = []
# for tweet_id in tweet_ids:
#     out = client.get_tweets(tweet_id, expansions='author_id', user_fields='id')
#     user_id = out.includes.get('users')[0] if out.includes.get('users') else None
#     if user_id:
#         users.append(user_id.get('id'))
#     else:
#         user.append(None)
# len(users)

In [ ]:
# Sample Detailed Tweet Lookup Code, given tweet-id

out = api.lookup_statuses(id=list(tweet_ids[:10]), map=True)
# out = api.get_status(','.join(tweet_ids[:10]))

followers = out[0]._json.get('user').get('followers_count') if out[0]._json.get('user') else None
verified = out[0]._json.get('user').get('verified') if out[0]._json.get('user') else None
geo = out[0]._json.get('geo').get('coordinates') if out[0]._json.get('geo') else None
geo = ','.join([str(g) for g in geo]) if geo else None
hashtags = out[0]._json.get('entities').get('hashtags') if out[0]._json.get('entities') else None
hashtags = ','.join([h.get('text') for h in hashtags])
created_at = out[0]._json.get('created_at')
user_created_at = out[0]._json.get('user').get('created_at') if out[0]._json.get('user') else None
retweet_count = out[0]._json.get('retweet_count')
lang = out[0]._json.get('lang', None)
tweet_id = out[0]._json.get('id_str')
tweet_text = out[0]._json.get('text')

followers, verified, geo, hashtags, created_at, user_created_at, retweet_count, lang, tweet_id, tweet_text

In [ ]:
# Prepare chunks for 100 tweets, as the api allows maximum 100 tweet-ids to be sent at once

chunk_size = 100
chunked_tweets = [tweet_ids[i:i + chunk_size] for i in range(0, len(tweet_ids), chunk_size)]

In [ ]:
# Function to prepare columns for our detailed tweet file by parsing api response

def decode_json(output):
    
    user_id = followers = verified = user_created_at = geo = hashtags = None
    
    tweet_created_at = output._json.get('created_at', None)
    retweet_count = output._json.get('retweet_count', None)
    lang = output._json.get('lang', None)
    tweet_id = output._json.get('id_str', None)
    tweet_text = output._json.get('text')
    
    time = tweets_dict.get(tweet_id)
    
    if output._json and output._json.get('user'):
        user = output._json.get('user')
        user_id = user.get('id', None)
        followers = user.get('followers_count', None)
        verified = user.get('verified', None)
        user_created_at = user.get('created_at', None)
        
    if output._json and output._json.get('geo'):
        geo = output._json.get('geo').get('coordinates', None)
        geo = ','.join([str(g) for g in geo]) if geo else None
        
    if output._json.get('entities'):
        hashtags = output._json.get('entities').get('hashtags')
        hashtags = ','.join([h.get('text') for h in hashtags]) if hashtags else None
    
    if not user_id:
        return None
        
    return {'Tweet': tweet_text, 'Tweet_id': tweet_id, 'User_id': user_id, 'User_created_at': user_created_at,
           'Followers': followers, 'User_verfied': verified, 'Geo': geo, 'Hashtags': hashtags, 
           'retweet_count': retweet_count, 'lang': lang, 'Tweet_created_at': tweet_created_at, "Time(Month-Year)": time}

In [ ]:
# Driver function to do api call and save parsed responses in file

i = 0
unrecognized_tweets = []
file_name = 'NEW_ORLEANS_Detailed.csv'

for tweet_ids in chunked_tweets:
    print("loop {} out of {}".format(i, len(chunked_tweets)))
    try:
        out = api.lookup_statuses(list(tweet_ids), map=True)
        for output in out:
            row = decode_json(output)
            if not row:
                unrecognized_tweets.append(output)
                continue   #Skip tweet which doesn't have any information being retrieved from tweet_id
            df = pd.DataFrame(row, index=[1])
            df.to_csv(file_name, mode='a', index=False, header=False)
    except Exception as e:
        print("{}\n".format(e))
    i += 1

city_data = pd.read_csv('~/Documents/Practice/NEW_ORLEANS_Detailed.csv', header=None, dtype=str)
Columns = ["Tweet", "Tweet-id", "User_id", "User_created_at", "Followers", "User_verfied", "Geo",
           "Hashtags", "retweet_count", "lang", "Tweet_created_at", "Time(Month-Year)"]
city_data.columns = Columns
city_data.to_csv(file_name, index = False, header=True)